In this notebook, we will try to build a very simple model to predict the outcome of tennis matched.  This is an ELO model.  Specifically, we will model

$$p(y_1) = \sigma(\beta_1 - \beta_2)$$, where $$\beta_1$$ and $$\beta_2$$ are coefficients for players 1 and 2 respectively

In [1]:
from models.logit.base import get_match_result_data

df, player_mapping, inverse_player_mapping = get_match_result_data()

In [2]:
df.shape

(50784, 62)

50k matches to work with

#### Set up training data

First, we'll just try training on everything in this time period and see what happens.  Later, we will split into train and test sets and figure out appropriate discounting, etc.

In [3]:
from models.logit.base import get_X_y

X, y = get_X_y(df, player_mapping, include_ranks=False)
assert ((X > 0).sum(axis=1) == 1).all()
assert ((X < 0).sum(axis=1) == 1).all()

#### Create Logistic Regression with NonZero Prior

I'll start with a zero prior and go from there...

In [ ]:
import pandas as pd

def describe_result(coefs, inverse_player_map):
    coef_df = pd.DataFrame(list(enumerate(coefs[:len(inverse_player_map)])),
        columns=['player_idx', 'coef']
    )
    coef_df['player_name'] = coef_df['player_idx'].map(lambda x: inverse_player_map[x])
    return coef_df.sort_values(by='coef', ascending=False)

In [ ]:
from ml.prior_logit import NonZeroLogit

nzl = NonZeroLogit(lmbda=1., seed=10)
nzl.fit(X, y)
describe_result(nzl.beta, inverse_player_mapping).head()

In [ ]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(fit_intercept=False)
logit.fit(X, y)
describe_result(logit.coef_[0], inverse_player_mapping).head()

#### Tune Time-Decay and Regularization

There is an argument that we should not be regularizing towards 0.  Players that appear infrequently are probably wors -- they qualify for fewer tournaments and progress to later rounds of tournaments less often.  For now however, we will continue to regularize towards 0.  Here, we investigate the following:

* What time decay weight is most predictive?
* How much regularization should we use?

In [ ]:
# Start by updating model every New Year's Day
BURN_IN_DATE = 2010
END_DATE = 2019
TIME_BREAKS = ['%d-01-01' % year for year in range(BURN_IN_DATE, END_DATE)]
TIME_BREAKS = map(pd.to_datetime, TIME_BREAKS)
TIME_BREAKS

In [ ]:
import numpy as np

def fit_model(train_X, train_y, test_X, test_y, weights=None, lmbda=1.0, prior=0.):
    print 'Fitting model...'
    logit = NonZeroLogit(lmbda=lmbda, prior=prior)
    if weights is None:
        weights = np.ones(train_X.shape[0])
    logit.fit(train_X, train_y, sample_weight=weights)
    print 'Predicting probs...'
    preds = logit.predict_proba(test_X)[:, 1] 
    print 'Predicting classes...'
    class_preds = logit.predict(test_X)
    return logit, preds, class_preds

Let's confirm that get_weights is doing what it's supposed to be doing:

In [ ]:
from models.logit.base import _get_weights

weights = _get_weights(df['date'].max(), df, 365.)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(df['date'], weights)

Looks like the weighting is doing roughly what it is supposed to...

In [ ]:
from sklearn.metrics import roc_auc_score

def eval_model(lmbda=1.0, halflife=None, prior=0., max_rank=None, include_ranks=False):
    if max_rank is not None:
        _df = df[
            (df['wrank'] < max_rank) &
            (df['lrank'] < max_rank)
        ].copy()
    else:
        _df = df.copy()
        
    mods = []
    out = []
    _df['__weight__'] = 10000 *_get_weights(
        _df['date'].max(),
        _df,
        halflife
    )
    for i in range(len(TIME_BREAKS)):
        cur_break = TIME_BREAKS[i]
        if i + 1 == len(TIME_BREAKS):
            next_break = pd.to_datetime('2099-12-31')
        else:
            next_break = TIME_BREAKS[i + 1]
        print cur_break, next_break
        train_df = _df[_df['date'] < cur_break]
        test_df = _df[
            (_df['date'] >= cur_break) &
            (_df['date'] < next_break)
        ]
        train_X, train_y = get_X_y(train_df, player_mapping, include_ranks=include_ranks)
        test_X, test_y = get_X_y(test_df, player_mapping, include_ranks=include_ranks)
        mod, preds, class_preds = fit_model(
            train_X,
            train_y,
            test_X,
            test_y,
            lmbda=lmbda,
            weights=train_df['__weight__']
        )
        mods.append(mod)
        out.extend(zip(test_df['match_id'].tolist(), preds.tolist(), class_preds.tolist()))
    out_df = pd.DataFrame(out, columns=['match_id', 'pred', 'class_pred'])
    final_df = pd.merge(_df, out_df, on='match_id')

    if max_rank is not None:
        rel = final_df[
            (final_df['lrank'] < max_rank) &
            (final_df['wrank'] < max_rank)
        ]
    else:
        rel = final_df

    auc = roc_auc_score(rel['y'], rel['pred'])
    accuracy = (rel['y'] == rel['class_pred']).mean() 
    print auc, accuracy
    return mods, out, final_df, auc, accuracy 

In [ ]:
N_ATTEMPTS = 20
lmbdas = np.power(10., np.random.uniform(-3, 3, size=N_ATTEMPTS))
hls = np.random.uniform(100, 4 * 365, size=N_ATTEMPTS)
priors = np.random.uniform(-5., 0., size=N_ATTEMPTS)
out = []
for lmbda, hl, prior in zip(lmbdas, hls, priors):
    print lmbda, hl, prior
    _, _, _, auc, accuracy = eval_model(lmbda=lmbda, halflife=hl, prior=prior)
    out.append((lmbda, hl, prior, auc, accuracy))

In [ ]:
tuning_df = pd.DataFrame(
    out,
    columns=[
        'lambda',
        'halflife',
        'prior',
        'auc',
        'accuracy'
    ]
)

In [ ]:
tuning_df.sort('auc', ascending=False)

In [ ]:
mods, _, final_df, _, _ = eval_model(lmbda=100.0, halflife=365, prior=-2.0, max_rank=None, include_ranks=False)

In [ ]:
final_df[['tournament', 'winner', 'loser', 'winner_idx', 'loser_idx', 'p1_idx', 'p2_idx', 'pred', 'y']].tail(20)

In [ ]:
for tb, mod in zip(TIME_BREAKS, mods):
    print '\n' + str(tb)
    print '\n---------------------------'
    print describe_result(mod.beta, inverse_player_mapping).head(20)

How are we calibrated?

In [ ]:
from sklearn.calibration import calibration_curve

_x, _y = calibration_curve(final_df['y'], final_df['pred'])
plt.plot(_x, _y)
plt.plot(_x, _x)

Pessimistic on the lower end, optimistic on the higher end...

In [ ]:
def investigate_player(player_name):
    return df[
        df['winner'].map(lambda x: player_name in x) |
        df['loser'].map(lambda x: player_name in x)
    ][[
        'date',
        'winner',
        'loser',
    ]].tail(30)

In [ ]:
investigate_player('Roddick')

#### Naive Model that only uses rank to make class predictions

In [ ]:
df['rank_pred'] = (df['p1_rank'] < df['p2_rank'])
rel_df = df[
    (df['date'] > TIME_BREAKS[0]) &
    df['p1_rank'].notnull() &
    df['p2_rank'].notnull()
]
(rel_df['rank_pred'] == rel_df['y']).mean()

66.6% from using rank alone...but this_updates_constantly and this only looks at matches where both players have rankings...

In [ ]:
# Get year end rankings
rank_df = df.copy()
rank_df['year'] = rank_df['date'].map(lambda x: x.year)
_winner_rank_df = rank_df.groupby(['year', 'winner']).apply(
    lambda x: x[['year', 'winner', 'date', 'wrank']].iloc[-1]
).reset_index(drop=True)
_winner_rank_df.rename(columns={'winner': 'player'}, inplace=True)
_winner_rank_df.rename(columns={'wrank': 'ye_rank'}, inplace=True)

_loser_rank_df = rank_df.groupby(['year', 'loser']).apply(
    lambda x: x[['year', 'loser', 'date', 'lrank']].iloc[-1]
).reset_index(drop=True)
_loser_rank_df.rename(columns={'loser': 'player'}, inplace=True)
_loser_rank_df.rename(columns={'lrank': 'ye_rank'}, inplace=True)

tot_rank_df = pd.concat([_winner_rank_df, _loser_rank_df])
tot_rank_df.sort('date', ascending=True, inplace=True)
year_end_rankings = tot_rank_df.drop_duplicates(['player', 'year'], take_last=True)

In [ ]:
_for_merge = year_end_rankings.copy()
_for_merge['year'] += 1  # Want to match year end rankings for 2000 with matches in 2001

In [ ]:
rank_df = pd.merge(
    rank_df,
    _for_merge,
    left_on=['winner', 'year'],
    right_on=['player', 'year']
)
rank_df.rename(columns={'ye_rank': 'winner_ye_rank'}, inplace=True)

rank_df = pd.merge(
    rank_df,
    _for_merge,
    left_on=['loser', 'year'],
    right_on=['player', 'year']
)
rank_df.rename(columns={'ye_rank': 'loser_ye_rank'}, inplace=True)

In [ ]:
rel_df = rank_df[
    (rank_df['date'] > TIME_BREAKS[0]) &
    rank_df['winner_ye_rank'].notnull() &
    rank_df['loser_ye_rank'].notnull()
]
(rank_df['winner_ye_rank'] < rank_df['loser_ye_rank']).mean()

Only 64% accuracy when we use year-end rankings so apples-to-apples time comparison and can't update rank all the time